# Testing file reading

In [9]:
with open('../res/channelnames_funnyplaylist.txt') as f:
    f.write()
    # youtube_funny_list = f.read().splitlines()[2:]

In [24]:
with open('../res/channelnames_funnyplaylist.txt', 'a') as f:
    f.write('\nhello')

In [12]:
with open('../res/channelnames_funnyplaylist.txt', 'w') as f:
    f.write('yoyoyo')

In [30]:
with open('../res/channelnames_funnyplaylist.txt', 'r') as f:
    filecontents = f.read().splitlines()

with open('../res/channelnames_funnyplaylist.txt', 'w') as f:
    f.write('\n'.join(filecontents[:-1]))

In [32]:
'\n'.join(filecontents[:-1])

'https://www.youtube.com/playlist?list=PLtfd05dSaGmiMHvhB4fIqMWTj7zVzEJt0\n\npubg-mobile\nback-it-up-terry\nreverse-the-vehicle-terrence\nbitconnect\nwhai\ni-dont-care-that-you-broke-your-elbow\nwhose-toes-are-those\nhe-disconnected\nthere-are-8-days-in-a-week\noh-shiver-me-timbers\nthats-a-ten\ni-love-refrigerators\nskin-tone-chicken-bone\ncan-you-tell-the-time\nim-brian\nmy-love-for-you-is-like-diarrhea\nthere-is-no-fridge\ncat-hits-juul\njesus-christ-its-jason-bourne\nthis-guy-moaned-at-least-this-loud\n846-minutes-to-get-no-questions-right\nmy-teacher-answered-lol\neric-andre-danny-devito\nchonk-chart\noh-lawd-he-comin\nare-you-gettin-rude-to-man\nbop-thats-what-you-get-fam\nwhos-endz-you-comin-to-now-fam\nim-with-the-science-team\nhow-tall-are-you-yogurt\nyour-moms-a-hoe\nwent-to-africa-a-week-ago\ni-aint-never-heard-a-fart-hit-3rd-gear\nclown-juice\nthey-gave-me-clown-juice\noh-no-fart\nshrub\nbeeeeean\na-mimir\nprotactinium\ncarlos\ntravis-scott-yeeaeuh\nritz-car\ngtfo-outta-her

# Somethin from before July 2021

In [2]:
f = open('../src/audio/sphinx_en-US/pronounciation-dictionary.dict', 'r')
f = f.read()

In [8]:
words = f.split('\n')

In [27]:
words_arr = [word.split(' ', 1) for word in words][:-1]

In [30]:
matching = []
for word, phonemes in words_arr:
    if phonemes[-2:] == 'ER': matching.append(word)

In [31]:
len(matching)

10035

In [36]:
new_matching = []
for word in matching:
    if '(' not in word: new_matching.append(word)

In [41]:
out = open('../res/erwords.txt', 'w')

In [42]:
for word in new_matching:
    out.write(word + '\n')

In [49]:
www = open('../res/erwords.txt', 'r').read().split()

In [55]:
'for' in www

False

In [48]:
for word in matching:
    if len(word) < 3: print(word)

er
jr
mr
sr
ur


In [57]:
longest = ''
for word in www:
    if len(longest) < len(word):
        print(word)
        longest = word

aachener
accelerator
accelerometer
airconditioner
cinematographer
crystallographer
great-grandfather
